In [1]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets 
from torchvision import transforms
from torchvision.utils import save_image
from torch.autograd import Variable


import pandas as pd 
import numpy as np
import os 
import glob
np.random.seed(1337)  # for reproducibility
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import torch.optim as optim
import torch
import torch.nn.functional as nn
import torch.autograd as autograd
import torch.optim as optim
import numpy as np
#import matplotlib.pyplot as plt
#import matplotlib.gridspec as gridspec
import os
from torch.autograd import Variable
from sklearn.utils import shuffle
import random
from keras.datasets import mnist
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
from keras import optimizers

Using TensorFlow backend.


In [2]:
train_lre = pd.read_csv('/home/satishk/lre2.0/ivectors_csv_revised/train_feat_BNF_h5_07Nov_Shreyas.csv')
#train_afds

In [3]:
#train_lre = train_lre.iloc[1000:2000]

In [4]:
train_lre.head()

,0,1,2,3,4,5,6,7,8,9,...,491,492,493,494,495,496,497,498,499,langid
0,-0.141175,-1.160019,0.891814,1.898842,0.393065,0.983582,-0.559143,-0.761900,0.525598,0.344597,...,1.587687,-0.609223,1.529694,1.677775,-0.388426,1.044584,-1.365691,-0.354752,-0.815562,spa-eur
1,0.971834,0.035743,0.785385,1.328749,-0.043342,1.219254,0.761090,1.069464,0.879541,0.898939,...,-1.208031,0.527433,-0.709180,-1.117489,0.566520,1.642208,-0.703815,0.376027,-1.425985,spa-eur
2,-0.315703,-0.868423,0.619893,1.717784,-0.846024,1.177214,-0.191977,-0.658569,0.529625,0.313590,...,-0.896515,-0.128051,-0.075781,1.307904,-1.443047,2.551083,0.436309,-0.623076,0.194851,spa-eur
3,0.247949,-0.279720,1.320115,1.772152,-0.130394,0.517343,-0.290516,0.406225,-0.833366,1.567592,...,-0.398762,-0.078148,0.168076,-0.272796,-1.102862,0.358274,0.090417,-0.490244,1.819196,spa-eur
4,-0.986578,-0.357548,1.550832,1.400735,0.444350,0.465660,0.209248,1.175847,-1.079963,0.270915,...,0.947946,0.917957,-1.043352,0.414811,0.016396,-0.485602,-0.433002,-0.432070,0.778250,spa-eur


In [5]:
train_lre.groupby(['langid']).size()

langid
ara-acm    1406
ara-apc    3509
ara-ary     919
ara-arz     440
eng-gbr      98
eng-usg    2448
por-brz     444
qsl-pol     587
qsl-rus    1221
spa-car     688
spa-eur     121
spa-lac     898
zho-cmn    3331
zho-nan      95
dtype: int64

In [6]:
val_lre = pd.read_csv('/home/satishk/lre2.0/ivectors_csv_revised/dev_feat_BNF_h5_07Nov_Shreyas.csv')


In [7]:
#val_lre = val_lre.iloc[100:300]

In [8]:
val_lre.head()

,segmentid,language_code,data_source,speech_duration,0,1,2,3,4,5,...,491,492,493,494,495,496,497,498,499,uttid
0,lre17_ntrlosgu.sph,ara-acm,mls14,30,1.697234,0.029428,-0.400756,0.513963,-0.939232,1.500797,...,-1.314428,-0.927694,-0.370424,-0.514735,1.290885,0.688205,-0.494330,-0.053206,-1.330860,lre17_ntrlosgu
1,lre17_moxnwuqe.sph,ara-acm,mls14,10,1.648232,-0.053318,-0.562867,1.035870,-1.577741,1.593584,...,-0.929262,-1.301574,2.034934,-0.226545,-0.198926,-0.116174,0.347923,-0.870801,-2.599601,lre17_moxnwuqe
2,lre17_meesvkxz.sph,ara-acm,mls14,3,1.242829,0.675515,-0.371491,0.534970,-0.246783,0.806262,...,0.691336,0.257988,1.058771,1.018635,-1.929319,-0.307404,-0.486431,-2.839053,-2.704527,lre17_meesvkxz
3,lre17_rqmsmzui.sph,ara-acm,mls14,30,1.226681,0.014810,-0.396915,-0.097507,-0.013574,1.087025,...,1.049862,0.285627,2.385587,0.680073,1.500978,1.660566,-0.370672,-0.924109,0.096676,lre17_rqmsmzui
4,lre17_qgszpuyw.sph,ara-acm,mls14,10,1.411728,-0.119300,0.136256,0.030535,-1.029447,1.227100,...,0.155196,-1.030222,2.933880,-1.417872,-0.227513,0.748810,-0.671044,0.595977,1.722917,lre17_qgszpuyw


In [9]:
X_train = train_lre.drop("langid",axis=1)
y_train = train_lre["langid"]
#y_train_uttid = train_lre["uttid"]

In [10]:
X_val = val_lre.drop(["language_code","uttid","segmentid","data_source","speech_duration"],axis=1)
y_val = val_lre["language_code"]
y_val_segmentid = val_lre["segmentid"]

In [11]:
X_train = X_train.append(X_val, ignore_index=True)
y_train = y_train.append(y_val, ignore_index=True)

In [12]:
le = preprocessing.LabelEncoder()
le.fit(y_train)

LabelEncoder()

In [13]:
le.classes_

array(['ara-acm', 'ara-apc', 'ara-ary', 'ara-arz', 'eng-gbr', 'eng-usg',
       'por-brz', 'qsl-pol', 'qsl-rus', 'spa-car', 'spa-eur', 'spa-lac',
       'zho-cmn', 'zho-nan'], dtype=object)

In [14]:
y_train=le.transform(y_train)

In [15]:
le = preprocessing.LabelEncoder()
le.fit(y_val)

LabelEncoder()

In [16]:
le.classes_

array(['ara-acm', 'ara-apc', 'ara-ary', 'ara-arz', 'eng-gbr', 'eng-usg',
       'por-brz', 'qsl-pol', 'qsl-rus', 'spa-car', 'spa-eur', 'spa-lac',
       'zho-cmn', 'zho-nan'], dtype=object)

In [17]:
y_val_labels = le.transform(y_val)

In [18]:
y_val_labels[0:100]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0])

In [19]:
X_train=X_train.values
X_val=X_val.values

In [20]:
nb_classes = 14

In [21]:
def indices_to_one_hot(data, nb_classes):
    """Convert an iterable of indices to one-hot encoded labels."""
    targets = np.array(data).reshape(-1)
    return np.eye(nb_classes)[targets]

In [22]:
Y_train = indices_to_one_hot(y_train, 14)
#Y_test = indices_to_one_hot(y_test, 14)
Y_val = indices_to_one_hot(y_val_labels, 14)

In [23]:
Y_val

array([[ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  1.],
       [ 0.,  0.,  0., ...,  0.,  0.,  1.],
       [ 0.,  0.,  0., ...,  0.,  0.,  1.]])

In [24]:
Y_train

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  1.],
       [ 0.,  0.,  0., ...,  0.,  0.,  1.],
       [ 0.,  0.,  0., ...,  0.,  0.,  1.]])

In [25]:
Y_val.shape

(3661, 14)

In [26]:
Y_train.shape

(19866, 14)

In [27]:
X_train = X_train.astype('float32')
#X_test = X_test.astype('float32')
X_val = X_val.astype('float32')

#X_train /= 255
#X_test /= 255
#X_val /= 255

print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
#print(X_test.shape[0], 'test samples')
print(X_val.shape[0], 'val samples')


X_train shape: (19866, 500)
19866 train samples
3661 val samples


In [28]:
X_train.shape

(19866, 500)

In [29]:
y_train.shape

(19866,)

In [30]:
#X_train,  y_train = shuffle(X_train, y_train, random_state=0)

In [31]:
X_val.shape

(3661, 500)

In [32]:
y_val_labels

array([ 0,  0,  0, ..., 13, 13, 13])

In [33]:
#X_val,  y_val_labels = shuffle(X_val, y_val_labels, random_state=0)

In [34]:
X_train, X_test, Y_train, y_test = train_test_split(X_train, Y_train, test_size=0.2, random_state=42)

In [35]:
#mnist = input_data.read_data_sets('/home/satishk/depy_04_AUG/MNIST_data', one_hot=True)
mb_size = 256
Z_dim = 100
X_dim = 500 #mnist.train.images.shape[1]
y_dim = 14 #mnist.train.labels.shape[1]
h_dim = 128
cnt = 0
lr = 1e-3


In [36]:
X_dim, y_dim

(500, 14)

In [37]:
class Generator(torch.nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.fc1 = torch.nn.Linear(114, 512)
        self.fc2 = torch.nn.Linear(512,500)

    def forward(self, z, c):
        inputs = torch.cat([z, c], 1)
        return F.sigmoid(self.fc2(F.relu(self.fc1(inputs))))



In [38]:
class Discriminator(torch.nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        #self.inputs = torch.cat([z, c], 1)
        self.fc1 = torch.nn.Linear(514, 512)
        self.fc2 = torch.nn.Linear(512,1)
        
    def forward(self, X, c):
        inputs = torch.cat([X, c], 1)
        return F.sigmoid(self.fc2(F.relu(self.fc1(inputs))))


In [39]:
G = Generator()
D = Discriminator()

In [40]:
G

Generator(
  (fc1): Linear(in_features=114, out_features=512)
  (fc2): Linear(in_features=512, out_features=500)
)

In [41]:
D

Discriminator(
  (fc1): Linear(in_features=514, out_features=512)
  (fc2): Linear(in_features=512, out_features=1)
)

In [42]:
ones_label = Variable(torch.ones(mb_size))
zeros_label = Variable(torch.zeros(mb_size))

In [43]:
learning_rate = 0.0002
betas = (0.5, 0.999)

In [44]:
# Loss function
criterion = torch.nn.BCELoss()

# Optimizers
G_solver = torch.optim.Adam(G.parameters(), lr=learning_rate, betas=betas)
D_solver = torch.optim.Adam(D.parameters(), lr=learning_rate/2, betas=betas)

In [ ]:
len(X_train)

In [ ]:
batch_size = mb_size
# Start training
for epoch in range(10):
    
    

    print('epoch:',epoch)
    #for i in range(XX_train):
    # Build mini-batch dataset
    #batch_size = images.size(0)
    #images = to_var(images.view(batch_size, -1))

    it=0
    while it+batch_size < len(X_train) :
        

        start= it
        end= it + batch_size


        z = Variable(torch.randn(mb_size, Z_dim))
        X = X_train[start:end]

        c = Y_train[start:end]
        X = Variable(torch.from_numpy(X))
        c = Variable(torch.from_numpy(c.astype('float32')))

        # Dicriminator forward-loss-backward-update
        G_sample = G(z, c)
        D_real = D(X, c)
        D_fake = D(G_sample, c)

        D_loss_real = nn.binary_cross_entropy(D_real, ones_label)
        D_loss_fake = nn.binary_cross_entropy(D_fake, zeros_label)
        D_loss = D_loss_real + D_loss_fake

        D_loss.backward()
        D_solver.step()

        # Housekeeping - reset gradient
        D.zero_grad()

        # Generator forward-loss-backward-update
        z = Variable(torch.randn(mb_size, Z_dim))
        G_sample = G(z, c)
        D_fake = D(G_sample, c)

        G_loss = nn.binary_cross_entropy(D_fake, ones_label)

        G_loss.backward()
        G_solver.step()

        # Housekeeping - reset gradient
        D.zero_grad()

        #Print and plot every now and then
        #if it % 2 == 0:

        print('Iter-{}; D_loss: {}; G_loss: {}'.format(it, D_loss.data.numpy(), G_loss.data.numpy()))



        it+= batch_size
           

In [ ]:
df_SF=pd.DataFrame()
samples_per_class = 1000
#c = np.zeros(shape=[samples_per_class, y_dim], dtype='float32')
#c[:, np.random.randint(0, 10)] = 1.
for i in range(14):
    #print(i)
    c = np.zeros(shape=[samples_per_class, y_dim], dtype='float32')
    c[:, i] = 1.
    c_df=pd.DataFrame(c)
    df_SF = df_SF.append(c_df,ignore_index = True)
    
    

In [ ]:
c_df.shape

In [ ]:
df_SF.shape[0]

In [ ]:
c_gen = df_SF.values

In [ ]:
z_gen = Variable(torch.randn(df_SF.shape[0], Z_dim))

In [ ]:
z_gen.data.shape

In [ ]:
c_gen = Variable(torch.from_numpy(c_gen))

In [ ]:
samples = G(z_gen, c_gen)

In [ ]:
samples.data.shape

In [ ]:
#Taking the generated iVectors we will try to check the acc by MLP

In [ ]:
train_X1 = samples.data.numpy()

In [ ]:
type(train_X1)

In [ ]:
train_y1 = c_gen.data.numpy()

In [ ]:
type(train_y1)

In [ ]:
train_y1

In [ ]:
X_train = pd.DataFrame(X_train)
Y_train = pd.DataFrame(Y_train)


In [ ]:
train_X1 = pd.DataFrame(train_X1)
train_y1 = pd.DataFrame(train_y1)

In [ ]:
#train_y

In [ ]:
train_X = X_train.append(train_X1, ignore_index=True)
train_y = Y_train.append(train_y1, ignore_index=True)

In [ ]:
#train_X,  train_y = shuffle(train_X, train_y, random_state=0)

In [ ]:
train_X = train_X.values
train_y = train_y.values

In [ ]:
train_y

In [ ]:
model = Sequential()
model.add(Dense(512, input_dim=500))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(nb_classes))
model.add(Activation('softmax'))

In [ ]:
model.summary()

In [ ]:
adam=optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
best_weights_filepath = '/home/satishk/saved_weights/best_weights_2l_MLP_11.hdf5'
saveBestModel = ModelCheckpoint(best_weights_filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='auto')


In [ ]:
batch_size = 256
nb_epoch=30

In [ ]:
#Checking Accuracy with training+augmented data train_X and train_y are 'train + augmented' data
history = model.fit(train_X, train_y, batch_size=batch_size, epochs=nb_epoch,verbose=2, 
                    validation_data=(X_test , y_test),callbacks=[saveBestModel])

In [ ]:
#Frame label accuracy
score = model.evaluate(X_test, y_test, verbose=0)
print('ERROR after Data Augmentation %:', 1-score[1])